In [ ]:
# Extends to and tests out multivariate functionality and formatting of the variables in the model
# Combines synthetic data from mekiv_learning and some of our own synthetic data 

In [1]:
import torch
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

In [2]:
# Import one of our data sets to work with
dat = pd.read_csv('../causal-dim-reduction-text/synthetic_data/data/lda.1-1seed.128len.0.2-0.2delta.0.05-0.05tau.0.csv')

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dat.loc[:,dat.columns!='y'], 
                                                    dat.y, 
                                                    test_size=0.3, 
                                                    random_state=1234)


In [3]:
# LDA: N
# Lemmatization and removing stopwords
import nltk
# Lemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()
# Stopwords
stop_wrds = nltk.corpus.stopwords.words("english")
# Preprocess the data
# Convert all text to lower case
X_train.text = [txt.lower() for txt in X_train.text]
# Tokenize the words
processed_txt = X_train.text.apply(nltk.tokenize.word_tokenize) 
# Perform lemmatization
processed_txt = processed_txt.apply(lambda x: [lemmatizer.lemmatize(word) for word in x\
                                               if word not in stop_wrds])
# BOW representation
import gensim
# Create a dictionary from the processed text
dictionary_txt = gensim.corpora.Dictionary(processed_txt)
# Optional: filter out words that appear in fewer than 5 documents or more than 50% of documents
# dictionary_txt.filter_extremes(no_below = 5, no_above = 0.5)
# Create BOW representation
bow_corp = [dictionary_txt.doc2bow(txt) for txt in processed_txt]
# Train the LDA model
lda_model = gensim.models.LdaModel(corpus=bow_corp,
                                       id2word=dictionary_txt,
                                       num_topics=10)
# Obtain document over topics for new documents
# First preprocess the test documents
X_test.text = [txt.lower() for txt in X_test.text]
processed_test = X_test.text.apply(nltk.tokenize.word_tokenize)
processed_test = processed_test.apply(lambda x: [lemmatizer.lemmatize(word) for word in x\
                                                if word not in stop_wrds])
test_dict = gensim.corpora.Dictionary(processed_test)
bow_corp_test = [test_dict.doc2bow(txt) for txt in processed_test]
# Then obtain theta (list of tuples for each doc must be restructured)
test_docs_theta = [pd.DataFrame(lda_model.get_document_topics(doc_bow, minimum_probability=1e-7),\
                                columns = ['Topic', 'Proportion'])\
                   for doc_bow in bow_corp_test]
test_docs_theta = pd.concat(test_docs_theta)
test_docs_theta['Doc_idx'] = [idx for idx in processed_test.index for m in range(10)]
test_docs_theta = test_docs_theta.pivot(index = 'Doc_idx', columns='Topic', values='Proportion')


In [4]:
N = test_docs_theta.iloc[:,:-1] # Drop last column (not needed since rows sum to 1)

In [36]:
# Correlated topic models representation: M
import tomotopy as tp

# Stem using the porter stemmer 
porter_stem = nltk.PorterStemmer().stem
corp_txt = tp.utils.Corpus(tokenizer = tp.utils.SimpleTokenizer(porter_stem),
                          stopwords = stop_wrds)

corp_txt.process(txt.lower() for txt in X_train['text'].values.tolist())

# IDF: Inverse Document Frequency term weighting (term occurring in almost every document has very low weighting 
# and a term occurring at a few document has high weighting)

ctm_model = tp.CTModel(tw=tp.TermWeight.IDF, k=10, corpus=corp_txt)
# tp.CTModel(tw=tp.TermWeight.IDF, min_df=5, rm_top=40, k=10, corpus=corp_txt)

ctm_model.train(100)



In [101]:
# Then obtain theta (list of tuples for each doc must be restructured)
prep_ctm_test_docs = [ctm_model.make_doc([x for x in new_doc]) for new_doc in X_test['text']]
ctm_test_docs_theta = [ctm_model.infer(test_doc) for test_doc in prep_ctm_test_docs]


ctm_test_docs_theta = [pd.DataFrame({'Topic':pd.Series(range(1,11)),'Proportion': test_doc[0]}) \
                       for test_doc in ctm_test_docs_theta]
ctm_test_docs_theta = pd.concat(ctm_test_docs_theta)
ctm_test_docs_theta['Doc_idx'] = [idx for idx in processed_test.index for m in range(10)]
ctm_test_docs_theta = ctm_test_docs_theta.pivot(index = 'Doc_idx', columns='Topic', values='Proportion')

In [111]:
M = ctm_test_docs_theta.iloc[:,:-1] # Drop last column (not needed since rows sum to 1)

In [5]:
# # word2vec representation: M
# # Preprocessing
# # We can use gensim's built in preprocessing function simple_preprocess to lowercase and tokenize
# train_prep = [gensim.utils.simple_preprocess(txt) for txt in X_train.text]
# test_prep = [gensim.utils.simple_preprocess(txt) for txt in X_test.text]

# # Add tags for training data
# tagged_train = [gensim.models.doc2vec.TaggedDocument(doc, [i]) for i,doc in enumerate(train_prep)]
# # Create the model
# doc2vec_model = gensim.models.doc2vec.Doc2Vec(vector_size = 50, min_count = 2, epochs = 10)
# # Build vocabulary of model
# doc2vec_model.build_vocab(tagged_train)
# # Train the model
# doc2vec_model.train(tagged_train, total_examples = doc2vec_model.corpus_count,\
#                    epochs = doc2vec_model.epochs)
# # Predict/infer on test data using the model
# doc2vec_test = [doc2vec_model.infer_vector(txt) for txt in test_prep]
# # Convert to df
# doc2vec_test_df = pd.DataFrame(doc2vec_test)


In [6]:
# M = doc2vec_test_df


In [20]:
# Generate some data to be used as our variables for everything other than M and N
# Following sigmoid_design.py in the data folder

def f(x: np.ndarray) -> np.ndarray:
    return np.log(np.abs(16 * x - 8) + 1) * np.sign(x - 0.5)


# Train data set
mu = np.zeros((3,))
# mu = torch.zeros((3,))
sigma = np.array([[1, 0.5, 0], [0.5, 1, 0], [0, 0, 1]])
# sigma = torch.tensor([[1, 0.5, 0], [0.5, 1, 0], [0, 0, 1]])

# torch.distributions.MultivariateNormal(torch.zeros((3,)),\
#                                        torch.tensor([[1, 0.5, 0], [0.5, 1, 0], [0, 0, 1]])).sample()


from numpy.random import default_rng
# rng = default_rng(seed=rand_seed)
rng = default_rng(seed=1234)
data_size = M.shape[0]
utw = rng.multivariate_normal(mu, sigma, size=data_size*N.shape[1])
u = utw[:, 0:1]
from scipy import stats
z = stats.norm.cdf(utw[0:data_size, 2])[:, np.newaxis] 
Z=z
x = stats.norm.cdf(utw[:, 1] + utw[:, 2] / np.sqrt(2))[:, np.newaxis]
# x = z + rng.normal(0, 0.01, data_size)[:, np.newaxis]
X_hidden=x.reshape(-1, N.shape[1])
structural = f(x)
Y_struct=structural
outcome = f(x) + u
Y=outcome

# Let's use gaussian error
data_size = X_hidden.shape[0]
std_X = np.std(X_hidden)
# Select scale_m and scale_n
# scale_m = 0.25
# scale_n = 1
# std_M, std_N = std_X * scale_m, std_X * scale_n
# M = X_hidden + std_M * np.random.normal(0, 1, data_size)[:, np.newaxis]
# N = X_hidden + std_N * np.random.normal(0, 1, data_size)[:, np.newaxis]

covariate = None
X_obs = None


# Test data set
x_test = np.linspace(0, 1, 1000*N.shape[1])
y_test = f(x_test)
X_all_test = x_test.reshape(-1, N.shape[1])
Y_struct_test = y_test.reshape(-1, N.shape[1]).sum(axis=1)
covariate_test = None


In [118]:
x_test = np.linspace(0, 1, 1000*N.shape[1])
X_all_test = x_test.reshape(-1, N.shape[1])

In [126]:

Y_struct_test.shape

(1000,)

In [ ]:
# Mekiv method

In [22]:
# Preliminaries
train_params = {'split_ratio': 0.5, 
                'lambda_mn': [0, -10], 
                'lambda_n': [0, -10],
                'xi': [0, -10],
                'lambda_x': None,
                'n_chi': 500,
                'Chi_lim': [-0.5, 0.5],
                'label_cutoff': 1.0,
                'reg_param': 0.,
                'batch_size': 64, 
                'lr': 0.1, 
                'num_epochs': 5}

In [23]:
# ***BEGIN STAGE 1***
# Obtain training and testing indices
from sklearn.model_selection import train_test_split
trainset1_idx, trainset2_idx = train_test_split(np.arange(X_hidden.shape[0]),
                                                test_size = train_params['split_ratio'],
                                                random_state = 1234)
Z_trainset1 = Z[trainset1_idx] ; Z_trainset2 = Z[trainset2_idx]
M_trainset1 = M.iloc[trainset1_idx,:] ; M_trainset2 = M.iloc[trainset2_idx,:]
N_trainset1 = N.iloc[trainset1_idx,:] ; N_trainset2 = N.iloc[trainset2_idx,:]
X_hidden_trainset1 = X_hidden[trainset1_idx] ; X_hidden_trainset2 = X_hidden[trainset2_idx]
Y_trainset1 = Y[trainset1_idx] ; Y_trainset2 = Y[trainset2_idx]


In [25]:
# 2: obtain lambda and gamma via stage1_tuning function (trainer.py file)
from scipy.spatial.distance import cdist

# The stage1_tuning function is used to obtain gamma and lambda
# gamma_mn, lambda_mn = self.stage1_tuning(KMN1MN1, KMN1MN2, KZ1Z1, KZ1Z2, lambda_mn)
# Method is as follows
# Preliminaries
# Initialize lambda mn and lambda n
lambda_n = np.exp(np.linspace(train_params['lambda_n'][0], train_params['lambda_n'][1], 50))
lambda_mn = np.exp(np.linspace(train_params['lambda_mn'][0], train_params['lambda_mn'][1], 50))

# Obtain MN (concatenate along second axis)
MN_trainset1 = np.c_[M_trainset1, N_trainset1] ; MN_trainset2 = np.c_[M_trainset2, N_trainset2] 

sigmaN = np.median(cdist(N_trainset1, N_trainset1, "sqeuclidean"))
sigmaMN = np.median(cdist(MN_trainset1, MN_trainset1, "sqeuclidean"))
sigmaZ = np.median(cdist(Z_trainset1, Z_trainset1, "sqeuclidean"))

KZ1Z1 = np.exp(-cdist(Z_trainset1, Z_trainset1, "sqeuclidean") / 2 / float(sigmaZ))
# torch: KZ1Z1 = torch.exp(-torch.cdist(Z1, Z1, "sqeuclidean") / 2 / float(sigmaZ))
KZ1Z2 = np.exp(-cdist(Z_trainset1, Z_trainset2, "sqeuclidean") / 2 / float(sigmaZ))
KN1N1 = np.exp(-cdist(N_trainset1, N_trainset1, "sqeuclidean") / 2 / float(sigmaN))
KN1N2 = np.exp(-cdist(N_trainset1, N_trainset2, "sqeuclidean") / 2 / float(sigmaN))
KMN1MN1 = np.exp(-cdist(MN_trainset1, MN_trainset1, "sqeuclidean") / 2 / float(sigmaMN))
KMN1MN2 = np.exp(-cdist(MN_trainset1, MN_trainset2, "sqeuclidean") / 2 / float(sigmaMN))

# Calculation
n = Z_trainset1.shape[0]
# N
gamma_list = [np.linalg.solve(KZ1Z1 + n * lam1 * np.eye(n), KZ1Z2) for lam1 in lambda_n]
score = [np.trace(gamma.T.dot(KN1N1.dot(gamma)) - 2 * KN1N2.T.dot(gamma)) for gamma in gamma_list]
lambda_n = lambda_n[np.argmin(score)]
gamma_n = gamma_list[np.argmin(score)]
# MN
gamma_list = [np.linalg.solve(KZ1Z1 + n * lam1 * np.eye(n), KZ1Z2) for lam1 in lambda_mn]
score = [np.trace(gamma.T.dot(KMN1MN1.dot(gamma)) - 2 * KMN1MN2.T.dot(gamma)) for gamma in gamma_list]
lambda_mn = lambda_mn[np.argmin(score)]
gamma_mn = gamma_list[np.argmin(score)]

# ***END STAGE 1***

In [27]:
# ***BEGIN Merror STAGE***
# This is where the training happens with all the epochs and everything
# **1**
# The code uses the below function which we instead replace with its entire functionality
# stageM_data = create_stage_M_raw_data(self.n_chi, N1, M1, Z2, gamma_n, gamma_mn, sigmaN, KZ1Z2)

Chi_n = np.random.normal(0, 1, train_params['n_chi']* (N_trainset1.shape[1]))
Chi_n = Chi_n / 2 / np.pi / sigmaN ** 0.5 # because the computed sigmaN is actually sigma^2N
Chi_n = Chi_n.reshape(train_params['n_chi'],N_trainset1.shape[1])
n, m = KZ1Z2.shape
# Columns of Chi are repeated to account for the data size of the variable
cos_term = np.cos(Chi_n @ N_trainset1.T)  # shape: Chi.shape[0] x args.train.N.shape[0]
sin_term = np.sin(Chi_n @ N_trainset1.T)
# Real (cos) and imaginary (sin) parts; dot products with gamma N
denom = cos_term.dot(gamma_n) + sin_term.dot(gamma_n) * 1j 
# Component shape: Chi.shape[0] x args.dev.Z.shape[0]
m_gamma_numer = sum([gamma_mn * M_trainset1[i].to_numpy().reshape(-1,1) for i in range(M_trainset1.shape[1])])
numer = cos_term.dot(m_gamma_numer) + sin_term.dot(m_gamma_numer) * 1j 
raw_labels = (numer.to_numpy()/denom.to_numpy()).flatten().reshape(-1,1)
raw_Chi = np.repeat(Chi_n, m).reshape(-1, N_trainset1.shape[1])
raw_Z = np.repeat(Z_trainset2[np.newaxis, :, :], train_params['n_chi'], axis=0).reshape(-1, Z_trainset2.shape[1])
raw_dict = {'labels':raw_labels, 'Chi':raw_Chi, 'Z':raw_Z}


In [28]:
real_label = np.real(raw_dict['labels']).flatten()
imag_label = np.imag(raw_dict['labels']).flatten()
idx_select = (real_label < np.mean(real_label) + 1. * np.std(real_label)) * (
            real_label > np.mean(real_label) - 1. * np.std(real_label)) \
                 * (imag_label < np.mean(imag_label) + 1. * np.std(imag_label)) * (
                         imag_label > np.mean(imag_label) - 1. * np.std(imag_label))

In [29]:
# **2**
# The code uses the below function which we instead replace with its entire functionality
# stageM_data = prepare_stage_M_data(raw_data2=stageM_data, rand_seed=rand_seed)

real_label = np.real(raw_dict['labels']).flatten()
imag_label = np.imag(raw_dict['labels']).flatten()
idx_select = (real_label < np.mean(real_label) + 1. * np.std(real_label)) * (
            real_label > np.mean(real_label) - 1. * np.std(real_label)) \
                 * (imag_label < np.mean(imag_label) + 1. * np.std(imag_label)) * (
                         imag_label > np.mean(imag_label) - 1. * np.std(imag_label))
raw_labels = raw_dict['labels'][idx_select]
raw_Chi = raw_dict['Chi'][idx_select]
raw_Z = raw_dict['Z'][idx_select]
shuffle_idx = np.arange(raw_Z.shape[0])
default_rng(seed=1234).shuffle(shuffle_idx)
for key in raw_dict.keys():
    raw_dict[key][shuffle_idx]
# The below code line just converts the data to torch if needed then adds new values to the class
# Values added to class and converted to tensors
# Pretty sure this isn't needed though because all components are manually converted to tensors in the code 
# StageMDataSetTorch.from_numpy(raw_data2)
stageM_data = {'labels':raw_labels, 'Chi':raw_Chi, 'Z':raw_Z}
stage1_MNZ = {'M': M_trainset1.to_numpy(), 'N': N_trainset1.to_numpy(), 'Z': Z_trainset1, 'sigmaZ': sigmaZ}



In [ ]:
test_x_initialiser = (torch.tensor(stage1_MNZ['M'][:,0:stage1_MNZ['N'].shape[1]]) + torch.tensor(stage1_MNZ['N'])) / 2

In [90]:
test_KZ1Z1 = torch.tensor(np.exp(-cdist(stage1_MNZ['Z'], stage1_MNZ['Z'], "sqeuclidean") / 2 / float(stage1_MNZ['sigmaZ'])))
test_K_Z1z = torch.tensor(np.exp(-cdist(stage1_MNZ['Z'], stageM_data['Z'][1:300], "sqeuclidean") / 2 / float(stage1_MNZ['sigmaZ'])))
test_gamma_x_I_lambda = sum([torch.eye(stage1_MNZ['Z'].shape[0]) * torch.exp(test_x_initialiser[:,i].reshape(-1,1)) for i in range(test_x_initialiser.shape[1])])
test_gamma_x = torch.linalg.solve(test_KZ1Z1 + stage1_MNZ['Z'].shape[0] * test_gamma_x_I_lambda, test_K_Z1z)

In [91]:
# test_cos_term = torch.cos(torch.matmul(torch.tensor(stageM_data['Chi'][0:300].reshape(-1,1)).float(),test_x_initialiser.reshape(1, -1)))
# test_sin_term = torch.sin(torch.matmul(torch.tensor(stageM_data['Chi'][0:300].reshape(-1,1)).float(),test_x_initialiser.reshape(1, -1)))

test_cos_term = [torch.cos(torch.matmul(torch.tensor(stageM_data['Chi'][:,i].reshape(-1,1)[1:300]).float(),\
                                          test_x_initialiser[:,i].reshape(1, -1))) for i in range(test_x_initialiser.shape[1])]
test_sin_term = [torch.sin(torch.matmul(torch.tensor(stageM_data['Chi'][:,i].reshape(-1,1)[1:300]).float(),\
                                          test_x_initialiser[:,i].reshape(1, -1))) for i in range(test_x_initialiser.shape[1])]


In [93]:
# test_cos_w = torch.sum(test_cos_term * gamma_x.t(), dim=-1).reshape(-1, 1)
# test_sin_w = torch.sum(test_sin_term * gamma_x.t(), dim=-1).reshape(-1, 1)

test_cos_w = sum([torch.sum(test_cos_term[i] * test_gamma_x.t(), dim=-1).reshape(-1, 1)\
                                     for i in range(test_x_initialiser.shape[1])])
test_sin_w = sum([torch.sum(test_sin_term[i] * test_gamma_x.t(), dim=-1).reshape(-1, 1)\
                                     for i in range(test_x_initialiser.shape[1])])


In [103]:
test_cos_w_numer = sum([torch.sum(test_cos_term[i] * test_gamma_x.t() * test_x_initialiser[:,i].reshape(1, -1),\
                                               dim=-1).reshape(-1, 1) for i in range(test_x_initialiser.shape[1])])
test_sin_w_numer = sum([torch.sum(test_sin_term[i] * test_gamma_x.t() * test_x_initialiser[:,i].reshape(1, -1),\
                                          dim=-1).reshape(-1, 1) for i in range(test_x_initialiser.shape[1])])

In [104]:

# **3** 

# stage_m_out = self.stage_M_main(stageM_data=stageM_data, stage1_MNZ=stage1_MNZ, train_params=self.train_params)
class model_class(torch.nn.Module):
    def __init__(self, stageM_data: stageM_data, train_params: train_params, stage1_MNZ: stage1_MNZ,
                 gpu_flg: bool = False):
        super().__init__()
        self.stageM_data = stageM_data
        self.stage1_MNZ = stage1_MNZ
        self.reg_param = train_params['reg_param']
        # We are attempting to uncover a 1 dimensional X and thus initialize with the row averages of M and N 
#       self.x_initialiser = (torch.tensor(stage1_MNZ['M']).mean(axis=1) + torch.tensor(stage1_MNZ['N']).mean(axis=1)) / 2
        # Multidimensional X with the same dimensions as N    
        self.x_initialiser = (torch.tensor(stage1_MNZ['M'][:,0:stage1_MNZ['N'].shape[1]]) + torch.tensor(stage1_MNZ['N'])) / 2

        if not train_params['lambda_x']:
#             self.params = torch.nn.Parameter(self.x_initialiser.flatten())
#             self.x = self.params.reshape(-1,1)
            self.x = torch.nn.Parameter(self.x_initialiser)
            self.lambda_x = self.x
        else:
#             self.params = torch.nn.Parameter(self.x_initialiser.flatten())
#             self.x = self.params.reshape(-1,1)
            self.x = torch.nn.Parameter(self.x_initialiser)
            self.lambda_x = train_params['lambda_x']
        self.train_params = train_params
        self.KZ1Z1 = torch.tensor(np.exp(-cdist(stage1_MNZ['Z'], stage1_MNZ['Z'], "sqeuclidean") / 2 / float(stage1_MNZ['sigmaZ'])))
    def forward(self, idx):
        ### gamma ###
        n = self.stage1_MNZ['Z'].shape[0]
        z = self.stageM_data['Z'][idx]
        K_Z1z = torch.tensor(np.exp(-cdist(stage1_MNZ['Z'], z, "sqeuclidean") / 2 / float(stage1_MNZ['sigmaZ'])))
        # gamma = self.cme_X.brac_inv.matmul(K_Zz)
        if not self.train_params["lambda_x"]:
            gamma_x_I_lambda = sum([torch.eye(n) * torch.exp(self.lambda_x[:,i].reshape(-1,1)) for i in range(self.lambda_x.shape[1])])
            gamma_x = torch.linalg.solve(self.KZ1Z1 + n * gamma_x_I_lambda, K_Z1z)
#             gamma_x = torch.linalg.solve(self.KZ1Z1 + n * torch.exp(self.lambda_x) * torch.eye(n), K_Z1z)
            # gamma_x = torch.linalg.solve(self.KZ1Z1 + n * self.lambda_x * torch.eye(n), K_Z1z)
        else:
            gamma_x_I_lambda = sum([torch.eye(n) * torch.exp(self.lambda_x[:,i].reshape(-1,1)) for i in range(self.lambda_x.shape[1])])
            gamma_x = torch.linalg.solve(self.KZ1Z1 + n * gamma_x_I_lambda, K_Z1z)
#             gamma_x = torch.linalg.solve(self.KZ1Z1 + n * self.lambda_x * torch.eye(n), K_Z1z)


        ### decompose e^{i\mathcal{X}n_i} ###
        cos_term = [torch.cos(torch.matmul(torch.tensor(self.stageM_data['Chi'][:,i].reshape(-1,1)[idx]).float(),\
                                          self.x[:,i].reshape(1, -1))) for i in range(self.x.shape[1])]
        sin_term = [torch.sin(torch.matmul(torch.tensor(self.stageM_data['Chi'][:,i].reshape(-1,1)[idx]).float(),\
                                          self.x[:,i].reshape(1, -1))) for i in range(self.x.shape[1])]
#         cos_term = torch.cos(torch.matmul(torch.tensor(self.stageM_data['Chi'].reshape(-1,1)[idx]).float(),\
#                                           self.x.reshape(1, -1)))
#         sin_term = torch.sin(torch.matmul(torch.tensor(self.stageM_data['Chi'].reshape(-1,1)[idx]).float(),\
#                                           self.x.reshape(1, -1)))

        denom = {}
        # using gamma to evaluate the charasteristic function value at a bunch of curly_x's
        denom['cos_weighted'] = sum([torch.sum(cos_term[i] * gamma_x.t(), dim=-1).reshape(-1, 1)\
                                     for i in range(self.x.shape[1])])
        denom['sin_weighted'] = sum([torch.sum(sin_term[i] * gamma_x.t(), dim=-1).reshape(-1, 1)\
                                     for i in range(self.x.shape[1])])
#         denom['cos_weighted'] = torch.sum(cos_term * gamma_x.t(), dim=-1).reshape(-1, 1)
#         denom['sin_weighted'] = torch.sum(sin_term * gamma_x.t(), dim=-1).reshape(-1, 1)
        denom['value'] = denom['cos_weighted'] + denom['sin_weighted'] * 1j

        numer = {}
        numer['cos_weighted'] = sum([torch.sum(cos_term[i] * gamma_x.t() * self.x[:,i].reshape(1, -1),\
                                               dim=-1).reshape(-1, 1) for i in range(self.x.shape[1])])
        numer['sin_weighted'] = sum([torch.sum(sin_term[i] * gamma_x.t() * self.x[:,i].reshape(1, -1),\
                                          dim=-1).reshape(-1, 1) for i in range(self.x.shape[1])])
#         numer['cos_weighted'] = torch.sum(cos_term * gamma_x.t() * self.x.reshape(1, -1), dim=-1).reshape(-1, 1)
#         numer['sin_weighted'] = torch.sum(sin_term * gamma_x.t() * self.x.reshape(1, -1), dim=-1).reshape(-1, 1)
        numer['value'] = numer['cos_weighted'] + numer['sin_weighted'] * 1j

        return numer['value'] / denom['value']

model = model_class(stageM_data=stageM_data, train_params=train_params, stage1_MNZ=stage1_MNZ)

model.train() # tells your model that you are training the model, not evaluating it
optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

losses = []
early_stop = False
step = 0
for ep in range(train_params['num_epochs']):
    if early_stop:
        break
    running_loss = 0.0
    batches_idxes = []
    idxes = np.arange((stageM_data['Chi']).shape[0])
    np.random.shuffle(idxes)
    batch_i = 0
    while True:
        batches_idxes.append(torch.tensor(idxes[batch_i * train_params['batch_size']: (batch_i + 1) * train_params['batch_size']]))
        batch_i += 1
        if batch_i * train_params['batch_size'] >= (stageM_data['Chi']).shape[0]:
            break
    for i, batch_idx in enumerate(batches_idxes):
        preds = model(batch_idx)
        # Loss functionality
        labels = stageM_data['labels'][batch_idx]
        dim_label = labels.shape[-1]
        num_label = labels.shape[0]
        preds_as_real = torch.view_as_real(preds)
        labels_as_real = torch.view_as_real(torch.tensor(labels))
        mse = torch.sum((labels_as_real - preds_as_real) ** 2) / num_label / dim_label
#       reg = torch.sum((model.x - (torch.tensor(stage1_MNZ['M'].mean(axis=1) + stage1_MNZ['N'].mean(axis=1)) / 2)) ** 2)
        reg = torch.sum((model.x - (torch.tensor(stage1_MNZ['M'][:,0:stage1_MNZ['N'].shape[1]] + stage1_MNZ['N']) / 2))** 2)
        loss = mse + train_params['reg_param'] * reg

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 1 == 0:
            print('[epoch %d, batch %5d] loss: %.5f, mse: %.5f, reg: %.5f' % (
            ep + 1, i + 1, running_loss / 1, mse / 1, train_params['reg_param'] * reg / 1))
            running_loss = 0.0

        losses.append(loss.item())
        if step > 8000: # Max of 8000 iterations
            break
        if (step > 2) and np.abs(losses[-1] - losses[-2]) < 1e-7: # Convergence considered to be < 1e-7
            early_stop = True
            break
        step += 1


# Convert model to numpy after training
fitted_x = model.x.detach().numpy()
#  assert stage_M_out.fitted_x.shape[0] == stage1_MNZ.Z.shape[0]
if not train_params['lambda_x']:
    lambda_x = np.exp(model.lambda_x.detach().numpy())  # syntax?
else:
    lambda_x = model.lambda_x





[epoch 1, batch     1] loss: 0.97909, mse: 0.97909, reg: 0.00000
[epoch 1, batch     2] loss: 0.79093, mse: 0.79093, reg: 0.00000
[epoch 1, batch     3] loss: 0.62362, mse: 0.62362, reg: 0.00000
[epoch 1, batch     4] loss: 0.47784, mse: 0.47784, reg: 0.00000
[epoch 1, batch     5] loss: 0.35185, mse: 0.35185, reg: 0.00000
[epoch 1, batch     6] loss: 0.24783, mse: 0.24783, reg: 0.00000
[epoch 1, batch     7] loss: 0.16296, mse: 0.16296, reg: 0.00000
[epoch 1, batch     8] loss: 0.09829, mse: 0.09829, reg: 0.00000
[epoch 1, batch     9] loss: 0.05099, mse: 0.05099, reg: 0.00000
[epoch 1, batch    10] loss: 0.02131, mse: 0.02131, reg: 0.00000
[epoch 1, batch    11] loss: 0.00453, mse: 0.00453, reg: 0.00000
[epoch 1, batch    12] loss: 0.00014, mse: 0.00014, reg: 0.00000
[epoch 1, batch    13] loss: 0.00445, mse: 0.00445, reg: 0.00000
[epoch 1, batch    14] loss: 0.01488, mse: 0.01488, reg: 0.00000
[epoch 1, batch    15] loss: 0.02878, mse: 0.02878, reg: 0.00000
[epoch 1, batch    16] lo

In [112]:
# **4**
gamma_x = sum([np.linalg.solve(KZ1Z1 + n * lambda_x[:,i] * np.eye(n), KZ1Z2) for i in range(lambda_x.shape[1])])
#gamma_x = np.linalg.solve(KZ1Z1 + n * lambda_x * np.eye(n), KZ1Z2)
sigmaX = np.median(cdist(fitted_x, fitted_x, "sqeuclidean"))
KfittedX = np.exp(-cdist(fitted_x, fitted_x, "sqeuclidean") / 2 / float(sigmaX))
W = KfittedX.dot(gamma_x)

# ***END Merror STAGE***

In [113]:
# ***BEGIN STAGE 2***

xi = train_params['xi']
if isinstance(xi, list):
    xi = np.exp(np.linspace(xi[0], xi[1], 50))
    M = W.shape[1]
    b = W.dot(Y_trainset2)
    A = W.dot(W.T) # W.T is transpose of W
    alpha_list = [np.linalg.solve(A + M * lam2 * KfittedX, b) for lam2 in xi]
    score = [np.linalg.norm(Y_trainset1 - KfittedX.dot(alpha)) for alpha in alpha_list]
    alpha = alpha_list[np.argmin(score)]
    xi = xi[np.argmin(score)]
else:
    alpha = np.linalg.solve(W.dot(W.T) + m * self.xi * KfittedX, W.dot(Y_trainset2))
    
# ***END STAGE 2***


In [127]:
# ***OBTAIN FINAL OUTPUT***
# Concatenate the covariate with the test data if there is a covariate
if covariate_test is not None:
    X_all_test = np.concatenate([X_all_test, covariate_test], axis=-1)
# Obtain predictions 
Kx = np.exp(-cdist(X_all_test, fitted_x, "sqeuclidean") / 2 / float(sigmaX))
preds = np.dot(Kx, alpha)
# Evaluate the model 
mse = np.mean((Y_struct_test - preds)**2)


In [128]:
print(mse)

201.5096159526795


In [340]:
# Old mse (1 dimensional X)
print(mse)

2.4876152472680255
